In [1]:
import networkx as nx
import os
import osmnx as ox
import pandas as pd

ox.config(log_console=True)

graphs_folder = 'G:\\Geoff\\osmnx\\data\\tracts\\graphml'
indicators_path = 'data/tracts_indicators.csv'
elevation_folder = 'data/nodes_elevation'
output_path = 'data/tracts_grades.csv'

In [2]:
%%time
results = {}
for state_folder in os.listdir(graphs_folder):
    
    elevations = pd.read_csv(f'{elevation_folder}/{state_folder}.csv', index_col=0, header=None)[1]
    
    for filename in os.listdir(f'{graphs_folder}/{state_folder}'):
        
        filepath = f'{graphs_folder}/{state_folder}/{filename}'
        div = filepath.rfind('/') + 1
        folder = filepath[:div]
        filename = filepath[div:]
        geoid = filename.replace('.graphml', '')
        
        G = ox.load_graphml(filename=filename, folder=folder)
        
        graph_elevations = elevations.loc[set(G.nodes())]
        nx.set_node_attributes(G, name='elevation', values=graph_elevations)
        G = ox.add_edge_grades(G)

        grades = pd.Series(nx.get_edge_attributes(G, 'grade_abs'))
        
        el_iqr = graph_elevations.quantile(0.75) - graph_elevations.quantile(0.25)
        el_range = graph_elevations.max() - graph_elevations.min()
        results[geoid] = {'grade_mean' : grades.mean(),
                          'grade_median' : grades.median(),
                          'elevations_std' : graph_elevations.std(),
                          'elevations_range' : el_range,
                          'elevations_iqr' : el_iqr}

g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\elevation.py:119: RuntimeWarning: invalid value encountered in double_scalars
  grade = round(elevation_change / data['length'], 4)
G:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\elevation.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  grade = round(elevation_change / data['length'], 4)


Wall time: 2h 1min 45s


In [3]:
df = pd.DataFrame(results).T
df.head()

,elevations_iqr,elevations_range,elevations_std,grade_mean,grade_median
01001020100,47.33050,86.482,25.815171,0.024849,0.01740
01001020200,35.53200,77.211,24.781705,0.025185,0.01880
01001020300,17.92425,76.987,15.680329,0.025761,0.01765
01001020400,10.90325,39.386,7.819470,0.015734,0.01110
01001020500,7.87225,74.286,11.679680,0.014819,0.01020


In [4]:
df.to_csv(output_path, index=True, encoding='utf-8')

In [5]:
tracts = pd.read_csv(indicators_path, dtype={'geoid':str})
tracts = pd.merge(tracts, df, left_on='geoid', right_index=True)
len(tracts)

72663

In [6]:
tracts.to_csv('data/tracts_indicators_grades.csv', index=False, encoding='utf-8')